In [33]:
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import regex as re
from nltk.stem import PorterStemmer
from pandarallel import pandarallel
import itertools
import collections
import subprocess
import psutil
import os
from subprocess import Popen, PIPE, STDOUT

stemmer = PorterStemmer()

pandarallel.initialize(progress_bar = False)

# Serialization folder
SERIALIZATION_FOLDER = "pickle/"

# Serialization folder
DF_NAME = "df.pkl"

# Environment
DATA_PATH = 'data/'
EMAIL_DATA = 'Emails.csv'

ASUM_PATH = 'asum'

TOKENS_THLD = 15

FREQ_THLD = 10

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [34]:
df = pd.read_pickle(SERIALIZATION_FOLDER + DF_NAME)

print(f"Length before: {len(df)}")
df = df[df["Tokenized"].apply(lambda x: len(x)) > TOKENS_THLD]
print(f"Length after: {len(df)}")
df.head()

Length before: 6737
Length after: 1752


,SenderPersonId,MetadataDateSent,ExtractedSubject,ExtractedBodyText,DateYear,DateMonth,DateDay,ExtractedBodyTextCleaned,Tokenized
Id,,,,,,,,,
2,NaN,2011-03-03 05:00:00+00:00,NaN,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",2011,3,3,b6\nh: latest how syria is aiding qaddafi and ...,"[h, latest, syria, aiding, qaddafi, sid, hrc, ..."
6,80.0,2012-09-12 04:00:00+00:00,Meet The Right Wing Extremist Behind Anti-Musl...,Pis print.\n-•-...-^\nH < hrod17@clintonernail...,2012,9,12,°russorv@state.gov'\nfrom [meat)\nsent: to: 11...,"[state.gov', meat, sent, subject, meet, right,..."
14,10.0,2011-03-13 05:00:00+00:00,NaN,"Anne-Marie Slaughter\nSunday, March 13, 2011 9...",2011,3,13,"anne-marie slaughter\njacob mills, cheryl d; r...","[anne-marie, slaughter, jacob, mills, cheryl, ..."
15,32.0,2012-09-12 04:00:00+00:00,RE: Not a dry eye in NEA,"_ .....\nFrom Randolph, Lawrence M\nSent: Wedn...",2012,9,12,"_ .....\nfrom randolph, lawrence m\nsent: to: ...","[randolph, lawrence, sent, mills, cheryl, subj..."
16,77.0,2012-09-12 04:00:00+00:00,NaN,I asked to attend your svtc today with Embassy...,2012,9,12,i asked to attend your svtc today with embassy...,"[asked, attend, svtc, today, embassy, tripoli,..."


In [35]:
def tokenize_sentence(sentence, stem=True):
    tokenized = word_tokenize(sentence)
    # Strip tokens
    tokenized = [token.strip() for token in tokenized]
    # Strict regex rule
    tokenized = [token for token in tokenized if re.match('\w+', token)]
    # Remove punctuation
    tokenized = [token for token in tokenized if token not in string.punctuation]
    # Remove stopwords
    stop = stopwords.words('english') + [':', '.', '@'] + ["n't"]
    tokenized = [token for token in tokenized if token not in stop]
    # Remove numbers
    tokenized = [token for token in tokenized if not re.search(r'\d', token)]
    if stem:
        tokenized = [stemmer.stem(token) for token in tokenized]
    return tokenized

def preprocess_asum(body):
    sentences = sent_tokenize(body)
    sentences = [tokenize_sentence(sentence) for sentence in sentences]
    return [s for s in sentences if s != []]

In [36]:
text = df.iloc[4]['ExtractedBodyTextCleaned']
text

"i asked to attend your svtc today with embassy tripoli, because had first met so many of that staff when i went with\nyou from malta to tripoli for the reopening of our embassy.\ntoday's deaths hit me much harder than i would have guessed. i am always proud to serve under you, but never have\nyour words been more meaningful than on today's svtc. every day of your tenure has been extraordinary, but none\nmore so than today. thank you again for your inspirational leadership and example.\nas ever,\nharold\nu.s. department of state\ncase no. f-2015-04841\ndoc no. c05739571\ndate: 05/13/2015\nstate dept. - produced to house select benghazi comm.\nsubject to agreement on sensitive information & redactions. no foia waiver. state-scb0045269"

In [37]:
sent_text = nltk.sent_tokenize(text)
sent_text

['i asked to attend your svtc today with embassy tripoli, because had first met so many of that staff when i went with\nyou from malta to tripoli for the reopening of our embassy.',
 "today's deaths hit me much harder than i would have guessed.",
 "i am always proud to serve under you, but never have\nyour words been more meaningful than on today's svtc.",
 'every day of your tenure has been extraordinary, but none\nmore so than today.',
 'thank you again for your inspirational leadership and example.',
 'as ever,\nharold\nu.s. department of state\ncase no.',
 'f-2015-04841\ndoc no.',
 'c05739571\ndate: 05/13/2015\nstate dept.',
 '- produced to house select benghazi comm.',
 'subject to agreement on sensitive information & redactions.',
 'no foia waiver.',
 'state-scb0045269']

In [38]:
[tokenize_sentence(s) for s in sent_text]

[['ask',
  'attend',
  'svtc',
  'today',
  'embassi',
  'tripoli',
  'first',
  'met',
  'mani',
  'staff',
  'went',
  'malta',
  'tripoli',
  'reopen',
  'embassi'],
 ['today', 'death', 'hit', 'much', 'harder', 'would', 'guess'],
 ['alway', 'proud', 'serv', 'never', 'word', 'meaning', 'today', 'svtc'],
 ['everi', 'day', 'tenur', 'extraordinari', 'none', 'today'],
 ['thank', 'inspir', 'leadership', 'exampl'],
 ['ever', 'harold', 'u.s.', 'depart', 'state', 'case'],
 ['doc'],
 ['date', 'state', 'dept'],
 ['produc', 'hous', 'select', 'benghazi', 'comm'],
 ['subject', 'agreement', 'sensit', 'inform', 'redact'],
 ['foia', 'waiver'],
 []]

In [39]:
df["asum"] = df["ExtractedBodyTextCleaned"].parallel_apply(preprocess_asum)
df.head()

,SenderPersonId,MetadataDateSent,ExtractedSubject,ExtractedBodyText,DateYear,DateMonth,DateDay,ExtractedBodyTextCleaned,Tokenized,asum
Id,,,,,,,,,,
2,NaN,2011-03-03 05:00:00+00:00,NaN,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",2011,3,3,b6\nh: latest how syria is aiding qaddafi and ...,"[h, latest, syria, aiding, qaddafi, sid, hrc, ...","[[h, latest, syria, aid, qaddafi, sid, hrc, me..."
6,80.0,2012-09-12 04:00:00+00:00,Meet The Right Wing Extremist Behind Anti-Musl...,Pis print.\n-•-...-^\nH < hrod17@clintonernail...,2012,9,12,°russorv@state.gov'\nfrom [meat)\nsent: to: 11...,"[state.gov', meat, sent, subject, meet, right,...","[[state.gov', meat, sent, subject, meet, right..."
14,10.0,2011-03-13 05:00:00+00:00,NaN,"Anne-Marie Slaughter\nSunday, March 13, 2011 9...",2011,3,13,"anne-marie slaughter\njacob mills, cheryl d; r...","[anne-marie, slaughter, jacob, mills, cheryl, ...","[[anne-mari, slaughter, jacob, mill, cheryl, r..."
15,32.0,2012-09-12 04:00:00+00:00,RE: Not a dry eye in NEA,"_ .....\nFrom Randolph, Lawrence M\nSent: Wedn...",2012,9,12,"_ .....\nfrom randolph, lawrence m\nsent: to: ...","[randolph, lawrence, sent, mills, cheryl, subj...","[[randolph, lawrenc, sent, mill, cheryl, subje..."
16,77.0,2012-09-12 04:00:00+00:00,NaN,I asked to attend your svtc today with Embassy...,2012,9,12,i asked to attend your svtc today with embassy...,"[asked, attend, svtc, today, embassy, tripoli,...","[[ask, attend, svtc, today, embassi, tripoli, ..."


In [40]:
flatten = list(itertools.chain(*list(itertools.chain(*df["asum"]))))
tokens_freq = collections.Counter(flatten)
print("Describe tokens frequency")
pd.Series(tokens_freq.values()).describe()

Describe tokens frequency


count    20088.000000
mean        13.966697
std         57.815385
min          1.000000
25%          1.000000
50%          2.000000
75%          6.000000
max       2122.000000
dtype: float64

In [41]:
top_tokens = [token for token, occ in tokens_freq.items() if occ > FREQ_THLD]
print(f"After thld on frequency: {len(top_tokens)}")

After thld on frequency: 3419


In [42]:
top_tokens = [token for token in top_tokens if re.match("^[a-z]+$", token)]
top_tokens = set(top_tokens) - {'url', 'http'}
top_tokens = sorted(top_tokens)
top_tokens = {idx: token for idx, token in zip(top_tokens, range(len(top_tokens)))}
list(top_tokens.items())[:10]

[('abandon', 0),
 ('abba', 1),
 ('abc', 2),
 ('abedin', 3),
 ('abil', 4),
 ('abl', 5),
 ('abort', 6),
 ('abraham', 7),
 ('abroad', 8),
 ('absenc', 9)]

In [43]:
def resolve_asum(sentences):
    # Resolve indices
    resolved = []
    for sentence in sentences:
        resolved.append([top_tokens.get(token, None) for token in sentence])
    # Remove None values
    no_none = []
    for l in resolved:
        no_none.append([x for x in l if x is not None])
    # Remove empty list
    out_list = []
    for l in no_none:
        out_list.append([x for x in l if x != []])
    # Remove empty body
    return [x for x in out_list if x != []]

In [44]:
final_series = df["asum"].parallel_apply(resolve_asum)
final_series


Id
2       [[1336, 1694, 2944, 85, 2387, 2726, 1428, 1874...
6       [[2676, 2897, 1870, 2566, 3281, 1087, 286, 114...
14      [[2755, 1578, 1905, 487, 3, 2224, 1744, 161, 2...
15      [[2676, 1905, 487, 2897, 1476, 1910], [2506, 2...
16      [[201, 221, 3038, 960, 3091, 1158, 1885, 1819,...
                              ...                        
7934    [[933, 161, 330, 2082, 2320, 1867, 2354, 1476,...
7938    [[748, 1004, 2272, 1690, 2015, 3200, 1718, 140...
7939    [[2791, 467, 2662, 3224, 1371, 830, 1309, 2548...
7942    [[307, 468, 2234, 2671], [2671, 2489, 142, 177...
7944    [[2385, 2240, 1731, 1659, 1380, 782, 1353, 224...
Name: asum, Length: 1752, dtype: object

In [45]:
# Wordlist
with open(os.path.join(ASUM_PATH,'in', 'WordList.txt'), "w+") as f:
    for word in top_tokens.keys():
        f.write(word + "\n")

In [46]:
# Bag of sentences
with open(os.path.join(ASUM_PATH, 'in', 'BagOfSentences.txt'), "w+") as f:
    for senteces in final_series:
        f.write(str(len(senteces)) + "\n")
        for sentence in senteces:
            f.write(' '.join([str(x) for x in sentence]) + '\n')

In [47]:
# Run script

N_SENTIMENT = '2'
N_TOPICS = '7'
N_ITER = '1000'
N_THREAD = str(psutil.cpu_count())
ALPHA = '0.1'
BETHA = '0.001/0.1/0.1'
G = '1/1'
SCRIPT_NAME = 'run.sh'
SCRIPT_PATH = os.path.join(ASUM_PATH, 'bin')
INPUT = os.path.join(ASUM_PATH, 'in')
OUTPUT = os.path.join(ASUM_PATH, 'out')

# Get asum script path
pwd = subprocess.run("pwd", capture_output=True).stdout.decode().strip()
asum_script = os.path.join(pwd, SCRIPT_PATH)

# Build command
command = ' '.join([os.path.join(asum_script, SCRIPT_NAME), N_SENTIMENT, N_TOPICS, N_ITER, N_THREAD, ALPHA, BETHA, G])

In [48]:
print("Start script...\n")

#subprocess.run(command, shell=True, cwd=asum_script)
p = Popen(command, stdin=PIPE, stdout=PIPE, stderr=STDOUT, shell=True, cwd=asum_script)
for line in p.stdout:
    print(line.decode(), end='')
print("Script execution completed.")




Start script...

Documents: 1752
Unique Words: 3335
Documents: 1752
Unique Words: 3335
Topics: 7
Sentiments: 2 (dictionary: 2)
Alpha: 0.1
Beta: 
0.001 0.1 0.1 
Gamma: 1 1 
Iterations: 1000
Threads: 12
Input Dir: ../in
Dictionary Dir: ../in
Output Dir: ../out
Too Long Sentences: 132
Gibbs sampling started (Iterations: 1000, Threads: 12)
  - Iteration 0
amaz/20/0/ attract/35/0/ best/249/0/ comfort/29/0/ correct/26/0/ excel/23/0/ favorit/12/0/ fortun/16/0/ fun/11/0/ glad/15/0/ good/391/0/ great/211/0/ happi/85/0/ impress/27/0/ love/111/0/ nice/17/0/ perfect/21/0/ posit/221/0/ recommend/72/0/ satisfi/26/0/ thank/285/0/ worth/67/0/ 
bad/0/58/ complain/0/22/ complaint/0/22/ disappoint/0/20/ hate/0/18/ mess/0/10/ neg/0/29/ poor/0/43/ problem/0/182/ regret/0/14/ sorri/0/35/ terribl/0/12/ troubl/0/42/ unaccept/0/13/ unfortun/0/42/ upset/0/14/ wast/0/17/ worst/0/32/ wrong/0/56/ 
    Iteration 0 took 0.215s. (Estimated Time: 0h 3m)
  - Iteration 1
amaz/20/0/ attract/35/0/ best/249/0/ comfort/29/0